In [1]:
# importando bibliotecas necessárias

import requests
import pandas as pd
import pprint

In [2]:
# dados importantes

headers = {
    'Content-Type': 'application/json',
    'chave-api-dados': '249d215e1b35a32d5f9e0f90aadc9052'
}

In [50]:
# consultando as informações do órgãos siafi 'os'

pagina_os = 1
sem_dados_os = '[]'
df_os = pd.DataFrame()

response_os = requests.get('https://api.portaldatransparencia.gov.br/api-de-dados/orgaos-siafi?pagina={}'.format(pagina_os), 
                                headers = headers)
dados_os = response_os.json()

while response_os.text != sem_dados_os:
        response_os = requests.get('https://api.portaldatransparencia.gov.br/api-de-dados/orgaos-siafi?pagina={}'.format(pagina_os), 
                                        headers = headers)
        dados_os = response_os.json()
        df_consulta_os = pd.DataFrame(dados_os)
        df_os = pd.concat([df_os, df_consulta_os])
        pagina_os = pagina_os + 1

df_orgaos_siafi = df_os.reset_index()    
df_orgaos_siafi.to_csv('orgaos_siafi.csv', index = False)




In [43]:
# lista dos códigos dos órgãos
df_codigos_orgaos_siafi = df_orgaos_siafi['codigo']

# lista dos nomes dos órgãos
df_nomes_orgaos_siafi = df_orgaos_siafi['descricao']
display(df_codigos_orgaos_siafi)

0      01000
1      01901
2      02000
3      02001
4      02002
       ...  
587    97120
588    97220
589    97320
590    97400
591    99900
Name: codigo, Length: 592, dtype: object

In [ ]:
# consultando dados das notas fiscais por orgão 'nfpo'
pagina_nfpo = 1
contador = 0
sem_dados_nfpo = '[]'
df_nfpo = pd.DataFrame()

while df_codigos_orgaos_siafi[contador] != KeyError:
    response_nfpo = requests.get('https://api.portaldatransparencia.gov.br/api-de-dados/notas-fiscais?codigoOrgao={}&pagina={}'.format(df_codigos_orgaos_siafi[contador], pagina_nfpo), headers =  headers)
    
    if response_nfpo.text != sem_dados_nfpo:
        print('Consultando dados do órgão {} na página de nº {}'.format(df_nomes_orgaos_siafi[contador], pagina_nfpo))
        dados_nfpo = response_nfpo.json()
        df_nf_por_orgao = pd.DataFrame(dados_nfpo)
        df_nfpo = pd.concat([df_nfpo, df_nf_por_orgao])
        df_nfpo.reset_index()
        pagina_nfpo = pagina_nfpo + 1
    else:
        pagina_nfpo = 1
        contador = contador + 1
        
display(df_nfpo)
        